In [29]:
#importing required libraries
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import functools
import cv2
from PIL import Image
import numpy as np



In [44]:
def Preprocessing (image) : 
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    height, width = image.shape
    img = cv2.GaussianBlur(image, (9, 9), 0) #decrease noise for dialation
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 101, 30)
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 101, 30)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 20))
    img = cv2.dilate(img, kernel, iterations=8)
    contours = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0] 
    biggest_contour = functools.reduce(lambda c1, c2: c1 if cv2.contourArea(c1) > cv2.contourArea(c2) else c2,contours) #find the biggest contour for text area
    x, y, w, h = cv2.boundingRect(biggest_contour) # find smallest rect that can contain the text area after dialation
    image = image[y:y + h, x:x + w]
    return image


In [43]:
image = cv2.imread("..\\Data_split\\train\\Females\\F1.jpg")
img = Preprocessing(image)
resized_image = cv2.resize(image.astype('float'), (64, 128))
fd,hog_image = hog(resized_image,orientations=9,pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=True)
hog_image = np.array(hog_image)
hog_image = hog_image.ravel() 
print(hog_image)


[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def HOG (img):
    #It uses a detection window of 64x128 pixels, so the image is first converted into (64, 128) shape.
    resized_image = cv2.resize(image.astype('float'), (64, 128))
    #The image is then further divided into small parts, and then the gradient and orientation of each part is calculated
    #It is divided into 8x16 cells into blocks with 50% overlap
    #so there are going to be 7x15 = 105 blocks in total
    #each block consists of 2x2 cells with 8x8 pixels.
    #We take the 64 gradient vectors of each block (8x8 pixel cell) and put them into a 9-bin histogram
    #hog function parameters:
    #orientations: Number of bins in the histogram we want to create, the original research paper used 9 bins 
    #pixels_per_cell: Determines the size of the cell, as we mentioned earlier, it is 8x8.
    #cells_per_block: Number of cells per block, will be 2x2 as mentioned previously.
    #visualize: A boolean whether to return the image of the HOG, we set it to True
    fd,hog_image = hog(resized_image,orientations=9,pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=True)
    #convert hog_image to np array to use ravel() 
    hog_image = np.array(hog_image)
    #ravel(): is used to change a 2-dimensional array or a multi-dimensional array into a contiguous flattened array.
    hog_image = hog_image.ravel() 
    return hog_image